In [14]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [15]:
cwl_dir = '../cwl/'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [16]:
template = {
    'variant_filepath': {
        'class': 'File',
        'path': 'a/file/path'
    },
    
    'annotate_trials_keyword': 'a_string',
    'variant_file_type': 'a_string',
    'tumor_sample_name': 'a_string',
    'normal_sample_name': 'a_string'
}

In [17]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.druggability.yaml'), 'w'))

In [22]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    'id': 'druggability',
    'label': 'druggability',
    'baseCommand': ['/usr/bin/python', '/pecgs-druggability/src/run_druggability.py'],
    'inputs': [
        {
            'id': 'tumor_sample_name',
            'type': 'string',
            'inputBinding': {
                'position': 1
            }
        },
        {
            'id': 'variant_filepath',
            'type': 'File',
            'inputBinding': {
                'position': 2,
            }
        },
        {
            'id': 'variant_file_type',
            'type': 'string?',
            'default': 'maf',
            'inputBinding': {
                'position': 3
            }
        },
        {
            'id': 'normal_sample_name',
            'type': 'string',
            'inputBinding': {
                'prefix': '--normal-sample-name',
                'position': 0
            }
        },
        {
            'id': 'annotate_trials_keyword',
            'type': 'string?',
            'default': 'CHOL',
            'inputBinding': {
                'prefix': '--annotate-trials-keyword',
                'position': 0
            }
        },
        # needs path to be set so it works on compute1
        {
            'id': 'environ_PATH',
            'type': 'string?',
            'default': '/miniconda/envs/druggability/bin:$PATH'
        },
    ],
    'arguments': [
        {
            'position': 0,
            'prefix': '--output-dir',
            'valueFrom': 'output'
        },
        {
            'position': 0,
            'prefix': '--druggability-dir',
            'valueFrom': '/pecgs-druggability/src/druggability'
        }
    ],
    'outputs': [
        {
            'id': 'output',
            'type': 'File',
            'outputBinding': {'glob': 'output/output.txt'},
        },
        {
            'id': 'aux_trials_output',
            'type': 'File',
            'outputBinding': {'glob': 'output/aux_trials_output.txt'},
        },
    ],
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'estorrs/pecgs-neoscan:0.0.1'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 28000,
            'coresMin': 4
        },
        {
            'class': 'EnvVarRequirement',
            'envDef': {
                'PATH': '$(inputs.environ_PATH)',
            }
        }
    ]
}

In [23]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'druggability.cwl'), 'w'))